In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import TensorBoard

import numpy as np
import random
import io


In [2]:
path = keras.utils.get_file(
    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)


In [3]:
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Corpus length: 600893
Total chars: 56
Number of sequences: 200285


In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
#optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
optimizer='adam'
model.compile(loss="categorical_crossentropy", optimizer=optimizer)
tensor_board = TensorBoard('./logs/character_generation')


In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
epochs = 5
batch_size = 64

for epoch in range(epochs):
    print("Epoch: %d\n" % epoch)
    model.fit(x, y, batch_size=batch_size, epochs=1)

print()
print("Generating text after epoch: %d" % epoch)

start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print("...Diversity:", diversity)

    generated = ""
    sentence = text[start_index : start_index + maxlen]
    print('...Generating with seed: "' + sentence + '"')

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    print("...Generated: ", generated)
    print()

Epoch: 0

3130/3130 [==============================] - 19s 5ms/step - loss: 2.3841
Epoch: 1

3130/3130 [==============================] - 15s 5ms/step - loss: 2.0325
Epoch: 2

3130/3130 [==============================] - 15s 5ms/step - loss: 1.8861
Epoch: 3

3130/3130 [==============================] - 15s 5ms/step - loss: 1.7874
Epoch: 4

3130/3130 [==============================] - 15s 5ms/step - loss: 1.7151

Generating text after epoch: 4
...Diversity: 0.2
...Generating with seed: "em or world-pain in that which he finds "
...Generated:  of the some of the saint and the string the strong to the case of the supprisent and as a moral the still of the superses the string the sainity of the consentions of the consention of the expersed the sense of the supprisent of the world the restined to the consention of the saining the still of the was the sained the consiction of the will of the still of the same the sainity of the present of t

...Diversity: 0.5
...Generating with seed: "em or 

In [7]:
epochs = 10
batch_size = 64

for epoch in range(epochs):
    print("Epoch: %d\n" % epoch)
    model.fit(x, y, batch_size=batch_size, epochs=1)

print()
print("Generating text after epoch: %d" % epoch)

start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print("...Diversity:", diversity)

    generated = ""
    sentence = text[start_index : start_index + maxlen]
    print('...Generating with seed: "' + sentence + '"')

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    print("...Generated: ", generated)
    print()

Epoch: 0

3130/3130 [==============================] - 15s 5ms/step - loss: 1.6572
Epoch: 1

3130/3130 [==============================] - 15s 5ms/step - loss: 1.6116
Epoch: 2

3130/3130 [==============================] - 15s 5ms/step - loss: 1.5735
Epoch: 3

3130/3130 [==============================] - 15s 5ms/step - loss: 1.5413
Epoch: 4

3130/3130 [==============================] - 16s 5ms/step - loss: 1.5140
Epoch: 5

3130/3130 [==============================] - 15s 5ms/step - loss: 1.4903
Epoch: 6

3130/3130 [==============================] - 15s 5ms/step - loss: 1.4684
Epoch: 7

3130/3130 [==============================] - 15s 5ms/step - loss: 1.4476
Epoch: 8

3130/3130 [==============================] - 15s 5ms/step - loss: 1.4300
Epoch: 9

3130/3130 [==============================] - 15s 5ms/step - loss: 1.4148

Generating text after epoch: 9
...Diversity: 0.2
...Generating with seed: "t! a statesman who should do all this, w"
...Generated:  ith the strength of the same the dest